In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date

In [3]:
installs = pd.read_csv("installs.csv.gzip", compression='gzip')

/home/cecilia/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
installs.isnull().sum()

created                    0
application_id             0
ref_type                   0
ref_hash                   0
click_hash            480369
attributed                 0
implicit                   0
device_countrycode         0
device_brand          205068
device_model           26892
session_user_agent     14839
user_agent            150743
event_uuid            378343
kind                  378343
wifi                  186682
trans_id              472578
ip_address                 0
device_language        27577
dtype: int64

In [6]:
installs.shape

(481511, 18)

In [8]:
installs['application_id'].value_counts().head(10)

121    71487
36     64471
210    33672
14     25630
65     23067
68     18762
122    15170
155    13629
226    13469
21     12184
Name: application_id, dtype: int64

In [9]:
installs['ref_type'].value_counts()

1891515180541284343    398906
1494519392962156891     82605
Name: ref_type, dtype: int64

In [10]:
installs['attributed'].value_counts()

False    480369
True       1142
Name: attributed, dtype: int64

In [11]:
installs['implicit'].value_counts()

False    378343
True     103168
Name: implicit, dtype: int64

In [12]:
installs['device_countrycode'].value_counts()

6287817205707153877    481511
Name: device_countrycode, dtype: int64

In [14]:
installs['device_model'].value_counts().head(10)

6.794880e+18    46187
9.186120e+18    13271
5.660121e+18     8562
3.057402e+18     7384
7.346301e+18     6233
2.544397e+18     5635
5.127788e+18     5609
3.017401e+18     5297
9.173807e+18     5242
1.670346e+18     5158
Name: device_model, dtype: int64

In [16]:
installs['wifi'].value_counts()

True     235130
False     59699
Name: wifi, dtype: int64

In [19]:
installs['device_language'].value_counts().head(10)

6.977049e+18    233012
3.301378e+18    149638
8.441417e+18     46449
6.584214e+18      6041
4.060930e+18      5222
5.221863e+18      3867
1.774282e+18      2565
7.528974e+18      1778
3.491909e+18      1490
2.938044e+18       902
Name: device_language, dtype: int64

In [ ]:
def crear_features_installs(installs_entrenar,datetime_inicio):
    installs_entrenar['date'] = pd.to_datetime(installs_entrenar['date'])
    
    dia_en_segundos = 86400
    installs_entrenar['segundos'] = ((installs_entrenar['date']-datetime_inicio).dt.seconds + 
                                     (installs_entrenar['date']-datetime_inicio).dt.days*dia_en_segundos)

    
    # Cambio application_id por la probabilidad de que aparezca esa aplicacion
    aplicacion = events_entrenar['application_id'].value_counts(normalize=True)
    installs_entrenar['application_id'] = installs_entrenar['application_id'].replace(aplicacion)
        
    # Cambio device_model por la probabilidad de que aparezca ese modelo
    modelo = installs_entrenar['device_model'].value_counts(normalize=True)
    installs_entrenar['device_model'] = installs_entrenar['device_model'].replace(modelo)
    
    # Cambio device_language por la probabilidad de que aparezca ese lenguaje
    lenguaje = installs_entrenar['device_language'].value_counts(normalize=True)
    installs_entrenar['device_language'] = installs_entrenar['device_language'].replace(lenguaje)
    
    # Cambio ref_type por la probabilidad de que aparezca ese ref_type
    tipo = installs_entrenar['ref_type'].value_counts(normalize=True)
    installs_entrenar['ref_type'] = installs_entrenar['ref_type'].replace(tipo)
    

    # Agrupo el data frame por los ref_hash
    installs_entrenar = installs_entrenar.groupby('ref_hash').agg({'ref_hash':'count','attributed':'sum',
                                                               'wifi':'sum','implicit':'sum',
                                                               'application_id':'mean','device_model':'mean',
                                                               'device_language':'mean','ref_type':'mean',
                                                               'segundos':['min','mean','max']})
    
    installs_entrenar.columns = ['e_kind','e_app_id','e_event_id','e_#wifi','e_#atribuidos','e_#eventos','e_t_prom']
    installs_entrenar = installs_entrenar.reset_index('ref_hash')
          
    return installs_entrenar